In [41]:
# Import packages.
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os



In [42]:
# Create embeddings dataframe and labels.
embeddings = np.load("chest-xray14_embeddings.npy")
annotation = pd.read_csv('Data_Entry_2017.csv')
annotation = annotation.rename(columns={'Image Index': 'img_id', 'Finding Labels': 'class_name', 'Patient ID':'patient_id', 'Patient Age':'age', 'Patient Gender': 'gender', 'View Position': 'view_position'})
annotation['class_name'] = annotation['class_name'].str.split('|')
print(annotation)

                  img_id                 class_name  Follow-up #  patient_id  \
0       00000001_000.png             [Cardiomegaly]            0           1   
1       00000001_001.png  [Cardiomegaly, Emphysema]            1           1   
2       00000001_002.png   [Cardiomegaly, Effusion]            2           1   
3       00000002_000.png               [No Finding]            0           2   
4       00000003_000.png                   [Hernia]            0           3   
...                  ...                        ...          ...         ...   
112115  00030801_001.png          [Mass, Pneumonia]            1       30801   
112116  00030802_000.png               [No Finding]            0       30802   
112117  00030803_000.png               [No Finding]            0       30803   
112118  00030804_000.png               [No Finding]            0       30804   
112119  00030805_000.png               [No Finding]            0       30805   

        age gender view_position  Origi

In [43]:
# Create labels.
file_path = "selected_png_list.txt"
ids = []

# Open the list of embeddings and create a list.
with open(file_path, 'r') as file:
    for line in file:
        full_path = line.strip()  
        filename = os.path.basename(full_path)
        ids.append(filename)

# Map the img id to a class name.
label_dict = annotation.set_index('img_id')['class_name'].to_dict()


# If the label is no finding set the value to 0 otherwise set to 1.
labels = []
for id in ids:
    if label_dict[id] == ['No Finding']:
        labels.append(0)
    else:
        labels.append(1)

# Create a dataframe for the embeddings.
df = embeddings




In [44]:
# Create embeddings train/test split.
# Find the 80% index.
cutoff_inx = int(len(embeddings) * 0.8)

# Split the dataset.
X_train = embeddings[0:cutoff_inx]
X_test = embeddings[cutoff_inx:]
y_test = labels[cutoff_inx:]
y_train = labels[0:cutoff_inx]

# Print the embeddings.
print(embeddings)

[[-0.0538576   0.00995135  0.04935905 ... -0.05894173  0.00644073
   0.05660105]
 [-0.05293966 -0.05798699  0.01407015 ... -0.04542941  0.01265656
   0.02598063]
 [-0.01355517 -0.01268439  0.00450761 ... -0.05543687  0.0033815
   0.00534222]
 ...
 [-0.04429238 -0.02848177  0.00751584 ... -0.04399014  0.00893611
   0.02493958]
 [-0.05206261 -0.03824705  0.00059087 ... -0.07417466  0.04271731
   0.06842596]
 [-0.05088723 -0.00479658  0.03359561 ... -0.07803639  0.03270045
   0.05343036]]


## Logistic Regression Binary Classification
Instead of one hot encoding each possible outomce and doing a one vs rest logistic classifier, we want to do binary classification if there is a disease outcome or not. This should theoretically improve the accuracy of the model relative to performing OvR multiclassification.


First we perform logistic regression on the whole dataset. Essentially we aggregate all of the samples and see if we can perform logistic regression on the whole dataset.


In [45]:
# Undersampling for whole dataset.
def undersample(X, y):
    # Convert to numpy arrays for easier manipulation
    X = np.array(X)
    y = np.array(y)
    
    # Find the indices of each class
    class0_indices = np.where(y == 0)[0]
    class1_indices = np.where(y == 1)[0]
    
    # Find the number of instances in the smaller class
    n_min = min(len(class0_indices), len(class1_indices))
    
    # Randomly select n_min indices from each class
    np.random.shuffle(class0_indices)
    np.random.shuffle(class1_indices)
    class0_indices = class0_indices[:n_min]
    class1_indices = class1_indices[:n_min]
    
    # Concatenate the indices and extract the samples
    undersampled_indices = np.concatenate((class0_indices, class1_indices))
    X_undersampled = X[undersampled_indices]
    y_undersampled = y[undersampled_indices]
    
    return X_undersampled, y_undersampled



In [46]:
# Implement a sigmoid function.

def sigmoid(x):
    return 1 / (1 + np.exp(-x - 1))


# Compute cross entropy loss
def compute_loss(y, y_hat):
    # Ensure that both y and y_hat are NumPy arrays
    y = np.array(y)
    y_hat = np.array(y_hat)

    # Avoid division by zero and log domain issues
    epsilon = 1e-15
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)
    
    # Compute the binary cross-entropy loss
    loss = -np.mean(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
    return loss


# Predict function
def predict(X, weights, bias, threshold = 0.5):
    z = np.dot(X, weights) + bias
    y_pred = sigmoid(z)
    print(y_pred)
    return (y_pred >= threshold).astype(int)
    #return y_pred.round()  # returns class labels based on threshold of 0.5


# Implement gradient descent.
def grad_desc(X, y, num_iterations, learning_rate, lambda_param, decay):
    # Perform feature scaling
    X = (X - np.mean(X, axis=0)) / np.std(X, axis=0, ddof= 0)

    # Find the shape of X
    X = np.array(X)
    
    m, n = X.shape


    # Set the bias.
    bias = np.random.random()

    # Set weights.
    weights = (np.random.randn(n) * 2 - 1) * 0.05
    #weights = np.zeros(n)

    for i in range(num_iterations):


        # Do the linear combination of the weights plus biases.
        z = np.dot(X, weights) + bias
        # Do the prediction with the current weights.

        predicted = sigmoid(z)
        # Find change in weights. (Calculate the gradient).
        deltaweights = np.dot(X.T, (predicted - y)) / m + lambda_param * (weights) / m
        deltabias = np.sum(predicted - y) / m

        # Update the weights and biases.
        weights -= learning_rate*deltaweights
        bias -= learning_rate*deltabias

        loss = compute_loss(y, predicted)
        # Print out every one hundred steps.
        if num_iterations % 10 == 0.0:
            print(f"Step {i}, Loss: {loss}")
        learning_rate *= decay

    return weights, bias


# Precision, Recall, and F1 Score calculation
def calculate_precision_recall_f1(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)


    TP = np.sum((y_pred == 1) & (y_true == 1))
    FP = np.sum((y_pred == 1) & (y_true == 0))
    FN = np.sum((y_pred == 0) & (y_true == 1))
    TN = np.sum((y_pred == 0) & (y_true == 0))
    print(TP, FP, TN, FN)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    acc = (TP+TN) / (TP+FP+TN+FN)
    return precision, recall, f1_score, acc


def TestWeightsBiases(X_test, y_test, weights, biases):
    # Make predictions
    predictions = predict(X_test, weights, biases)
    # Evaluate precision, recall, and F1 score
    precision, recall, f1_score, acc = calculate_precision_recall_f1(y_test, predictions)

    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1_score:.2f}, Accuracy: {acc:.2f}")
    return acc, f1_score


In [47]:
# Undersample and balance the classes.
X_train, y_train = undersample(X_train, y_train)

In [48]:
weights, biases = grad_desc(X_train, y_train, 5000, 0.125, lambda_param=1e-10, decay = 0.9995)
TestWeightsBiases(X_test, y_test, weights, biases)

Step 0, Loss: 1.361913691635065
Step 1, Loss: 1.2458914083658899
Step 2, Loss: 1.1779128131973193
Step 3, Loss: 1.1315916295468138
Step 4, Loss: 1.0972735995949126
Step 5, Loss: 1.0702665779737621
Step 6, Loss: 1.04795405724573
Step 7, Loss: 1.0288164473358377
Step 8, Loss: 1.0119317931145748
Step 9, Loss: 0.9967136042491913
Step 10, Loss: 0.9827739604616035
Step 11, Loss: 0.9698489417312822
Step 12, Loss: 0.9577551722754499
Step 13, Loss: 0.946362892989253
Step 14, Loss: 0.9355785693911787
Step 15, Loss: 0.9253333888936488
Step 16, Loss: 0.9155755526508748
Step 17, Loss: 0.9062650671282656
Step 18, Loss: 0.8973702011251078
Step 19, Loss: 0.8888650601885939
Step 20, Loss: 0.8807279152399058
Step 21, Loss: 0.8729400435391769
Step 22, Loss: 0.865484920132774
Step 23, Loss: 0.8583476508364593
Step 24, Loss: 0.8515145728976641
Step 25, Loss: 0.8449729728610199
Step 26, Loss: 0.8387108868440218
Step 27, Loss: 0.8327169590356426
Step 28, Loss: 0.826980341469581
Step 29, Loss: 0.8214906231118

(0.5666666666666667, 0.1646586345381526)

# Binary Logistic Regression Classification for each label. 

In [49]:
import random
# Define an undersampling function.
def undersample(list1, list2):
    # Determine the size of the smaller list
    smaller_size = min(len(list1), len(list2))

    # Randomly sample from both lists to make them the same size
    new_list1 = random.sample(list1, smaller_size)
    new_list2 = random.sample(list2, smaller_size)

    return new_list1, new_list2

In [50]:
# Create a list of unique features.
classes = annotation["class_name"]
uniq_classes = set()
for multiclass in classes:
    for classification in multiclass:
        uniq_classes.add(classification)

uniq_classes = list(uniq_classes)
print(uniq_classes)
# Now we want to create a list for each type of class.
dataset_individual = [[]] * len(uniq_classes)

# Now we need a dictionary to map the class to the index.
map_individual_index = dict(zip(uniq_classes, range(0, len(uniq_classes))))
print(map_individual_index)
# We now want to populate each item of the dataset with our image id's if it belongs to that class.
for i, _ in enumerate(uniq_classes):
    # Get list of image id's for the ith disease state.
    class_ids = annotation[annotation['class_name'].apply(lambda x: uniq_classes[i] in x)]['img_id']
    class_ids = list(class_ids)
    class_ids = set(class_ids).intersection(set(ids))
    dataset_individual[i] = class_ids

# Find the no finding part of the array.
idx_no_finding = map_individual_index['No Finding']
no_findings = dataset_individual[idx_no_finding]

dataset_individual[idx_no_finding] = []

for i, _ in enumerate(uniq_classes):
    undersample_one, undersample_two = undersample(list(dataset_individual[i]), list(no_findings))

    dataset_individual[i] = list(undersample_two) + list(undersample_one)
    # Shuffle the new list
    random.shuffle(dataset_individual[i])

count = 0
for i in dataset_individual:
    print(len(i))
    count += len(i)

print(f"total length is now {count}")



['Nodule', 'Fibrosis', 'Effusion', 'Pneumothorax', 'Edema', 'Infiltration', 'Hernia', 'No Finding', 'Cardiomegaly', 'Mass', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Emphysema', 'Pleural_Thickening']
{'Nodule': 0, 'Fibrosis': 1, 'Effusion': 2, 'Pneumothorax': 3, 'Edema': 4, 'Infiltration': 5, 'Hernia': 6, 'No Finding': 7, 'Cardiomegaly': 8, 'Mass': 9, 'Consolidation': 10, 'Pneumonia': 11, 'Atelectasis': 12, 'Emphysema': 13, 'Pleural_Thickening': 14}
1068
254
2252
960
446
3320
54
0
454
984
774
246
1936
424
644
total length is now 13816


In [51]:
# Create labels for the dataset.
labels_individual = []
for key, val in map_individual_index.items():
    labels_per_ind = []
    for i in dataset_individual[val]:
        if annotation[annotation["img_id"] == i]["class_name"].iloc[0] == ["No Finding"]:
            labels_per_ind.append(1)
        else:
            labels_per_ind.append(0)
    labels_individual.append(labels_per_ind)

In [52]:
with open("selected_png_list.txt") as f:
    lines = f.readlines()
    lines = [line.split("/") for line in lines]
    lines = [line[-1].strip("\n") for line in lines]

dict_img_index = {}
for i, line in enumerate(lines):
    dict_img_index[line] = i



In [53]:
# Can optionally change embeddings depending if we want to do feature selection.
embeddings = np.load("95_variance_PC.npy") # Uncomment if you would like to do feature selection with PCA
#embeddings = np.load("lasso_reduced_embeddings.npy") # Uncomment if you would like to do the feature selection with lasso.

print(dict_img_index)
dataset_individual_embeddings = dataset_individual.copy()
# Now we want to construct the embeddings dataset for each classification.
for i, _ in enumerate(dataset_individual):
    for item in dataset_individual[i]:
        dataset_individual_embeddings[i] = [embeddings[dict_img_index[item]] for item in dataset_individual[i]] 
    



{'00004118_002.png': 0, '00005241_000.png': 1, '00005681_020.png': 2, '00006469_000.png': 3, '00005921_000.png': 4, '00006024_004.png': 5, '00004875_000.png': 6, '00003948_001.png': 7, '00005232_000.png': 8, '00005954_003.png': 9, '00005524_001.png': 10, '00005462_000.png': 11, '00006500_004.png': 12, '00005480_000.png': 13, '00006271_027.png': 14, '00006149_000.png': 15, '00004001_010.png': 16, '00004920_004.png': 17, '00006570_000.png': 18, '00005722_005.png': 19, '00004007_005.png': 20, '00005778_027.png': 21, '00006008_000.png': 22, '00004342_056.png': 23, '00004490_001.png': 24, '00006564_000.png': 25, '00005801_001.png': 26, '00005456_005.png': 27, '00006497_003.png': 28, '00004066_001.png': 29, '00005004_002.png': 30, '00003996_019.png': 31, '00006022_000.png': 32, '00004123_006.png': 33, '00004823_000.png': 34, '00006027_000.png': 35, '00004268_009.png': 36, '00006304_013.png': 37, '00003989_019.png': 38, '00005532_030.png': 39, '00005736_000.png': 40, '00003957_002.png': 41, '

In [54]:
for key, val in map_individual_index.items():
    print(key, len(dataset_individual_embeddings[val]), val)

Nodule 1068 0
Fibrosis 254 1
Effusion 2252 2
Pneumothorax 960 3
Edema 446 4
Infiltration 3320 5
Hernia 54 6
No Finding 0 7
Cardiomegaly 454 8
Mass 984 9
Consolidation 774 10
Pneumonia 246 11
Atelectasis 1936 12
Emphysema 424 13
Pleural_Thickening 644 14


In [57]:

# Map the indices to the class labels.
map_index_class = {value: key for key, value in map_individual_index.items()}

# Change class to predict depending on what logistic regression classification model you want to use.
class_to_predict = "Pneumonia"
i = map_individual_index[class_to_predict] # Set i based on class

# Create the train and test split.
idx_split = int(len(dataset_individual_embeddings[i]) * 0.8)
X_train = dataset_individual_embeddings[i][:idx_split]
X_test = dataset_individual_embeddings[i][idx_split+1:]
y_train = labels_individual[i][:idx_split]
y_test =labels_individual[i][idx_split+1:]


acc = 0.0
f1 = 0.0

# Try to create the model until better than a random classifier. If it doesn't get better than random it just breaks the loop.
count = 0 
while acc < 0.5:
    weights, biases = grad_desc(X_train, y_train, 5000, 0.125, lambda_param=1e-10, decay = 0.9995)
    acc, f1 = TestWeightsBiases(X_test, y_test, weights, biases)
    if count == 10:
        break
    count += 1


Step 0, Loss: 0.9415177278185791
Step 1, Loss: 0.90802123445743
Step 2, Loss: 0.8766644512708593
Step 3, Loss: 0.8473223707748885
Step 4, Loss: 0.819869626618051
Step 5, Loss: 0.7941832344105638
Step 6, Loss: 0.7701446479341395
Step 7, Loss: 0.7476409991907089
Step 8, Loss: 0.7265656472335725
Step 9, Loss: 0.7068182576966008
Step 10, Loss: 0.6883046204566733
Step 11, Loss: 0.6709363551983819
Step 12, Loss: 0.654630596567388
Step 13, Loss: 0.6393097069909223
Step 14, Loss: 0.6249010360613751
Step 15, Loss: 0.6113367265578987
Step 16, Loss: 0.5985535556123625
Step 17, Loss: 0.5864927937777884
Step 18, Loss: 0.5751000640740271
Step 19, Loss: 0.5643251864069341
Step 20, Loss: 0.5541219984609108
Step 21, Loss: 0.5444481504271309
Step 22, Loss: 0.5352648762607495
Step 23, Loss: 0.5265367477318736
Step 24, Loss: 0.5182314191950342
Step 25, Loss: 0.5103193710693238
Step 26, Loss: 0.5027736590224942
Step 27, Loss: 0.495569674312316
Step 28, Loss: 0.48868491906774775
Step 29, Loss: 0.48209879875